# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Packages

In [1]:
import turicreate
import numpy as np
import pandas as pd
from math import log, sqrt
import matplotlib.pyplot as plt

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('~/Desktop/09.data_analysis/ML/coursera_washington/module2_regression/data/home_data.sframe/')
sales.head(5)

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat
0,3,7.0,1180.0,0.0,1955.0,0.0,98178,47.51123398
0,3,7.0,2170.0,400.0,1951.0,1991.0,98125,47.72102274
0,3,6.0,770.0,0.0,1933.0,0.0,98028,47.73792661
0,5,7.0,1050.0,910.0,1965.0,0.0,98136,47.52082


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms'] * sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors'] * sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.308315     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.391443     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
model_all.coefficients[model_all.coefficients['value'] != 0 ]

name,index,value,stderr
(intercept),None,274873.05595049576,None
bathrooms,None,8468.531086910105,None
sqft_living,None,24.42072098244546,None
sqft_living_sqrt,None,350.06055338605546,None
grade,None,842.0680348976231,None
sqft_above,None,20.024722417091112,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

ANS : bathrooms, sqft_living, sqft_living_sqrt, grade, and sqft_above have been chosen

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [17]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [18]:
l1_penalty = np.logspace(1, 7, num=13)
RSS_total = {} #dict
for i in l1_penalty:
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, verbose=False,
                                               l2_penalty=0, l1_penalty=i)
    residuals = validation['price'] - model.predict(validation)
    rss_sum = sum(residuals**2)
    RSS_total[i] = rss_sum

In [22]:
print(min(RSS_total.items()))

(10.0, 625766285142461.2)


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

ANS : 10

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [23]:
model_new = turicreate.linear_regression.create(sales, target='price', features= all_features,
                                                validation_set=None,
                                                l2_penalty=0,l1_penalty=10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.266467     | 5757083.459662     | 280315.641755                   |

| 10        | 11       | 0.000002  | 0.355170     | 5364325.408911     | 265887.138813                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [24]:
print(len(model_new.coefficients[model_new.coefficients['value'] != 0 ]))

18


ANS : 18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [25]:
l1_penalty_values = np.logspace(8, 10, num=20)
nonzero_list = {}
for i in l1_penalty_values:
    models = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                 verbose=None, validation_set=None,
                                                 l2_penalty=0, l1_penalty = i)
    a = len(models.coefficients[models.coefficients['value'] != 0 ])
    nonzero_list[i] = a

In [26]:
nonzero_list #key: penalty_value value : numbers of nonzero weights

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [27]:
l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536

In [73]:
round(3792690190.7322536, -6)

3793000000.0

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [29]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [40]:
max_nonzeros = 7
rss_sum = {}
nonzero_list = {}
for i in l1_penalty_values:   
    the_model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                    validation_set=None, verbose=None,
                                                   l2_penalty=0, l1_penalty=i)
    residuals = validation['price'] - the_model.predict(validation)
    rss_sum[i] = sum(residuals**2)
    
    a = len(the_model.coefficients[the_model.coefficients['value'] != 0 ])
    nonzero_list[i] = a

In [51]:
max_ = []
for k , v in nonzero_list.items():
    if v == max_nonzeros :
        max_.append(k)

In [70]:
for k, v in rss_sum.items():
    if k == min(max_):
        print('the l1_penalty : {}, and its rss is {}'.format(k, v))

the l1_penalty : 3448968612.163437, and its rss is 1046937488751713.5


In [72]:
round(3448968612.163437, -6)

3449000000.0

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [59]:
final_model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                    validation_set=None, verbose=None,
                                                   l2_penalty=0, l1_penalty=min(max_))
final_model.coefficients[final_model.coefficients['value'] != 0]

name,index,value,stderr
(intercept),None,222253.1925443279,None
bedrooms,None,661.7227177822278,None
bathrooms,None,15873.957259268,None
sqft_living,None,32.41022145125974,None
sqft_living_sqrt,None,690.1147733133166,None
grade,None,2899.420269749878,None
sqft_above,None,30.011575302201138,None
